In [2]:
import pandas as pd
import re
import glob
import os

In [3]:
df = pd.read_csv('../../data_files/cleaned/matches/teams/team_stats_cleaned.csv')

In [6]:
df

,match_id,date,stat,home_value,away_value,home_team,away_team
0,NaN,Sunday May 4,general_Possession %,48.80,51.20,New York City,Cincinnati
1,NaN,Sunday May 4,general_Shots,19.00,13.00,New York City,Cincinnati
2,NaN,Sunday May 4,general_Shots on Goal,5.00,4.00,New York City,Cincinnati
3,NaN,Sunday May 4,general_Blocked Shots,7.00,4.00,New York City,Cincinnati
4,NaN,Sunday May 4,general_Total Passes,435.00,444.00,New York City,Cincinnati
...,...,...,...,...,...,...,...
58121,NaN,Saturday September 21 2024,possession_81_85,38.47,61.53,Columbus,Orlando
58122,NaN,Saturday September 21 2024,possession_86_90,62.83,37.17,Columbus,Orlando
58123,NaN,Saturday September 21 2024,NaN,2.10,2.70,Columbus,Orlando
58124,NaN,Saturday September 21 2024,NaN,12.00,12.00,Columbus,Orlando


In [ ]:
### pivot long to wide

dfp = df.copy()

In [12]:
dfp['match_id'] = (
    dfp['date'].astype(str).str.strip() + '_' +
    dfp['home_team'].astype(str).str.strip() + '_' +
    dfp['away_team'].astype(str).str.strip()
).str.replace(' ', '')

In [13]:
dfp['stat_home'] = dfp['stat'] + '_home'
dfp['stat_away'] = dfp['stat'] + '_away'

dfp

,match_id,date,stat,home_value,away_value,home_team,away_team,stat_home,stat_away
0,SundayMay4_NewYorkCity_Cincinnati,Sunday May 4,general_Possession %,48.80,51.20,New York City,Cincinnati,general_Possession %_home,general_Possession %_away
1,SundayMay4_NewYorkCity_Cincinnati,Sunday May 4,general_Shots,19.00,13.00,New York City,Cincinnati,general_Shots_home,general_Shots_away
2,SundayMay4_NewYorkCity_Cincinnati,Sunday May 4,general_Shots on Goal,5.00,4.00,New York City,Cincinnati,general_Shots on Goal_home,general_Shots on Goal_away
3,SundayMay4_NewYorkCity_Cincinnati,Sunday May 4,general_Blocked Shots,7.00,4.00,New York City,Cincinnati,general_Blocked Shots_home,general_Blocked Shots_away
4,SundayMay4_NewYorkCity_Cincinnati,Sunday May 4,general_Total Passes,435.00,444.00,New York City,Cincinnati,general_Total Passes_home,general_Total Passes_away
...,...,...,...,...,...,...,...,...,...
58121,SaturdaySeptember212024_Columbus_Orlando,Saturday September 21 2024,possession_81_85,38.47,61.53,Columbus,Orlando,possession_81_85_home,possession_81_85_away
58122,SaturdaySeptember212024_Columbus_Orlando,Saturday September 21 2024,possession_86_90,62.83,37.17,Columbus,Orlando,possession_86_90_home,possession_86_90_away
58123,SaturdaySeptember212024_Columbus_Orlando,Saturday September 21 2024,NaN,2.10,2.70,Columbus,Orlando,NaN,NaN
58124,SaturdaySeptember212024_Columbus_Orlando,Saturday September 21 2024,NaN,12.00,12.00,Columbus,Orlando,NaN,NaN


In [14]:
home_pivot = dfp.pivot_table(
    index=['match_id', 'date', 'home_team', 'away_team'],
    columns='stat_home',
    values='home_value'
)

away_pivot = dfp.pivot_table(
    index=['match_id', 'date', 'home_team', 'away_team'],
    columns='stat_away',
    values='away_value'
)

In [15]:
final = pd.concat([home_pivot, away_pivot], axis=1).reset_index()

In [16]:
final

,match_id,date,home_team,away_team,general_Aerial Duels Won_home,general_Blocked Shots_home,general_Blocked_home,general_Clean Sheets_home,general_Clearances_home,general_Corners_home,...,possession_76_80_away,possession_81_85_away,possession_86_90_away,shooting_Blocked_away,shooting_Goals_away,shooting_Off Target_away,shooting_On Target_away,xg_Shots On Target_away,xg_Shots_away,xg_Total Team XG_away
0,FridayJuly25_Columbus_Orlando,Friday July 25,Columbus,Orlando,6.0,4.0,4.0,0.0,6.0,9.0,...,61.16,43.51,34.91,2.0,3.0,2.0,6.0,6.0,10.0,2.0
1,FridayJuly25_Dallas_NewYorkCity,Friday July 25,Dallas,New York City,14.0,1.0,1.0,0.0,3.0,2.0,...,40.05,74.32,41.92,3.0,4.0,1.0,5.0,5.0,9.0,1.7
2,FridayJuly25_LAFC_Portland,Friday July 25,LAFC,Portland,8.0,4.0,4.0,0.0,3.0,4.0,...,35.72,16.70,55.20,1.0,1.0,3.0,4.0,4.0,8.0,0.4
3,FridayJuly25_NewEngland_Montréal,Friday July 25,New England,Montréal,8.0,9.0,9.0,0.0,8.0,3.0,...,42.86,23.66,32.48,3.0,3.0,2.0,5.0,5.0,11.0,1.5
4,FridayJuly25_SanDiego_Nashville,Friday July 25,San Diego,Nashville,10.0,2.0,2.0,0.0,5.0,2.0,...,45.27,33.57,42.54,3.0,0.0,1.0,4.0,4.0,9.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,WednesdaySeptember182024_SaltLake_Dallas,Wednesday September 18 2024,Salt Lake,Dallas,8.0,0.0,0.0,0.0,12.0,2.0,...,40.40,42.99,54.20,6.0,2.0,4.0,5.0,5.0,16.0,2.8
997,WednesdaySeptember182024_Seattle_SanJose,Wednesday September 18 2024,Seattle,San Jose,11.0,6.0,6.0,0.0,1.0,5.0,...,57.63,69.44,80.14,3.0,2.0,3.0,3.0,3.0,10.0,1.5
998,WednesdaySeptember182024_Toronto_Columbus,Wednesday September 18 2024,Toronto,Columbus,5.0,4.0,4.0,0.0,2.0,5.0,...,59.94,59.36,81.85,1.0,2.0,2.0,4.0,4.0,7.0,1.2
999,WednesdaySeptember24_NewYorkCity_Miami,Wednesday September 24,New York City,Miami,3.0,2.0,2.0,0.0,2.0,4.0,...,24.11,44.82,31.83,1.0,4.0,3.0,8.0,8.0,12.0,2.2


In [18]:
### simplify colnames
final.columns = [c.replace(' ', '_').replace('%', 'pct').replace('-', '_').lower() for c in final.columns]

final

,match_id,date,home_team,away_team,general_aerial_duels_won_home,general_blocked_shots_home,general_blocked_home,general_clean_sheets_home,general_clearances_home,general_corners_home,...,possession_76_80_away,possession_81_85_away,possession_86_90_away,shooting_blocked_away,shooting_goals_away,shooting_off_target_away,shooting_on_target_away,xg_shots_on_target_away,xg_shots_away,xg_total_team_xg_away
0,FridayJuly25_Columbus_Orlando,Friday July 25,Columbus,Orlando,6.0,4.0,4.0,0.0,6.0,9.0,...,61.16,43.51,34.91,2.0,3.0,2.0,6.0,6.0,10.0,2.0
1,FridayJuly25_Dallas_NewYorkCity,Friday July 25,Dallas,New York City,14.0,1.0,1.0,0.0,3.0,2.0,...,40.05,74.32,41.92,3.0,4.0,1.0,5.0,5.0,9.0,1.7
2,FridayJuly25_LAFC_Portland,Friday July 25,LAFC,Portland,8.0,4.0,4.0,0.0,3.0,4.0,...,35.72,16.70,55.20,1.0,1.0,3.0,4.0,4.0,8.0,0.4
3,FridayJuly25_NewEngland_Montréal,Friday July 25,New England,Montréal,8.0,9.0,9.0,0.0,8.0,3.0,...,42.86,23.66,32.48,3.0,3.0,2.0,5.0,5.0,11.0,1.5
4,FridayJuly25_SanDiego_Nashville,Friday July 25,San Diego,Nashville,10.0,2.0,2.0,0.0,5.0,2.0,...,45.27,33.57,42.54,3.0,0.0,1.0,4.0,4.0,9.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,WednesdaySeptember182024_SaltLake_Dallas,Wednesday September 18 2024,Salt Lake,Dallas,8.0,0.0,0.0,0.0,12.0,2.0,...,40.40,42.99,54.20,6.0,2.0,4.0,5.0,5.0,16.0,2.8
997,WednesdaySeptember182024_Seattle_SanJose,Wednesday September 18 2024,Seattle,San Jose,11.0,6.0,6.0,0.0,1.0,5.0,...,57.63,69.44,80.14,3.0,2.0,3.0,3.0,3.0,10.0,1.5
998,WednesdaySeptember182024_Toronto_Columbus,Wednesday September 18 2024,Toronto,Columbus,5.0,4.0,4.0,0.0,2.0,5.0,...,59.94,59.36,81.85,1.0,2.0,2.0,4.0,4.0,7.0,1.2
999,WednesdaySeptember24_NewYorkCity_Miami,Wednesday September 24,New York City,Miami,3.0,2.0,2.0,0.0,2.0,4.0,...,24.11,44.82,31.83,1.0,4.0,3.0,8.0,8.0,12.0,2.2


In [6]:
## create function to reframe stats

def reframe_stats(df):

    # regex: match 'cleaned_stats_' + home + 'vs' + away + '-' + date + '.csv'
    m = re.match(r"cleaned_stats_([a-z]{2,4})vs([a-z]{2,4})-(\d{2}-\d{2}-\d{4})\.csv", fname, re.I)

    if m:
        home, away, date_str = m.groups()
        home, away = home.upper(), away.upper()
        date = pd.to_datetime(date_str, format="%m-%d-%Y")

        # add rows with stat = date and team = teams
        add_df = pd.DataFrame({
            'home_value': [home],
            'away_value': [away],
            'stat': 'teams'
        })
        
        new_row = pd.DataFrame({
            'home_value': [date],
            'away_value': [date],
            'stat': 'date'
        })

    df = pd.concat([df, add_df], ignore_index=True)

    df = pd.concat([df, new_row], ignore_index=True)
    out = {}
    for _, row in df.iterrows():
        out[f"{row['stat']}_home"] = row['home_value']
        out[f"{row['stat']}_away"] = row['away_value']

    wide = pd.DataFrame([out])
    wide= wide.drop(columns=['date_home'])
    wide['match_date'] = wide['date_away']
    wide['match_date'] = pd.to_datetime(wide['match_date'])
    wide = wide.drop(columns=['date_away'])
    wide.columns = [c.replace(' ', '_').replace('%', 'pct').replace('-', '_').lower() for c in wide.columns]
    return wide



In [18]:
idk2

,shooting_possession_pct_home,shooting_possession_pct_away,shooting_shots_home,shooting_shots_away,shooting_shots_on_goal_home,shooting_shots_on_goal_away,shooting_blocked_shots_home,shooting_blocked_shots_away,shooting_total_passes_home,shooting_total_passes_away,...,possession_86_90_away,xg_total_team_xg_home,xg_total_team_xg_away,xg_shots_home,xg_shots_away,xg_shots_on_target_home,xg_shots_on_target_away,teams_home,teams_away,match_date
0,58.1,41.9,12.0,8.0,4.0,2.0,1.0,2.0,601.0,377.0,...,11.67,2.0,0.5,12.0,8.0,4.0,2.0,SD,ATX,2025-05-31


In [19]:
import glob
import os

In [20]:
for file in glob.glob('*.csv'):
    filename = os.path.basename(file)
    df = pd.read_csv(file)
    cleaned_df = reframe_stats(filename, df)
    cleaned_df.to_csv(
        f"G:/My Drive/GitHubProjects/MLS/data/data_clean/matches/raw/reframed_stats/{filename}",
        index=False)

In [2]:
def reframe_stats(df):
    processed_matches = []
    
    for match_id, match_df in df.groupby('match_id'):
        match_record = {}

        match_record['match_id'] = match_id
        if 'date' in match_df.columns:
             match_record['match_date'] = match_df['date'].iloc[0]
        
        for _, row in match_df.iterrows():
            if pd.notna(row['stat_name']):
                stat_clean = row['stat_name'].replace(' ', '_').replace('%', 'pct').replace('-', '_').lower()
                
                match_record[f"{stat_clean}_home"] = row['home_value']
                match_record[f"{stat_clean}_away"] = row['away_value']
        
        processed_matches.append(match_record)
        
    wide = pd.DataFrame(processed_matches)
    
    return wide

In [3]:
df = pd.read_csv('../../data_files/scraped_raw/matches/teams/mls_match_team_stats.csv')


In [4]:
df

,stat_name,home_value,away_value,category,tip_id,home_possession,home_advantage,away_possession,away_advantage,match_id,date
0,Possession %,48.8,51.2,general,NaN,NaN,NaN,NaN,NaN,919a513b,Sunday May 4
1,Shots,19.0,13.0,general,NaN,NaN,NaN,NaN,NaN,919a513b,Sunday May 4
2,Shots on Goal,5.0,4.0,general,NaN,NaN,NaN,NaN,NaN,919a513b,Sunday May 4
3,Blocked Shots,7.0,4.0,general,NaN,NaN,NaN,NaN,NaN,919a513b,Sunday May 4
4,Total Passes,435.0,444.0,general,NaN,NaN,NaN,NaN,NaN,919a513b,Sunday May 4
...,...,...,...,...,...,...,...,...,...,...,...
579885,NaN,NaN,NaN,possession,bar_2_7,Home Possession: 70.94%,Home Advantage: 41.88%,Away Possession: 29.06%,Away Advantage: N/A,d63a2051,Saturday May 11 2024
579886,NaN,NaN,NaN,possession,bar_2_8,Home Possession: 66.87%,Home Advantage: 33.74%,Away Possession: 33.13%,Away Advantage: N/A,d63a2051,Saturday May 11 2024
579887,Total Team XG,3.2,1.2,xg,NaN,NaN,NaN,NaN,NaN,d63a2051,Saturday May 11 2024
579888,Shots,26.0,12.0,xg,NaN,NaN,NaN,NaN,NaN,d63a2051,Saturday May 11 2024


In [7]:
wide_df = reframe_stats(df)

NameError: name 'fname' is not defined

In [7]:
wide_df

,match_id,match_date,possession_pct_home,possession_pct_away,shots_home,shots_away,shots_on_goal_home,shots_on_goal_away,blocked_shots_home,blocked_shots_away,...,clean_sheets_home,clean_sheets_away,xg_conceded_home,xg_conceded_away,interceptions_home,interceptions_away,total_team_xg_home,total_team_xg_away,shots_on_target_home,shots_on_target_away
0,00093bf5,Saturday August 16,50.7,49.3,19.0,13.0,5.0,3.0,7.0,3.0,...,0.0,0.0,1.5,2.5,2.0,2.0,2.5,1.5,5.0,3.0
1,006b3126,Saturday October 11,57.1,42.9,20.0,10.0,10.0,3.0,5.0,2.0,...,0.0,0.0,1.3,2.8,0.0,3.0,2.8,1.3,10.0,3.0
2,00ba135d,Saturday May 4 2024,56.7,43.3,16.0,15.0,7.0,5.0,4.0,3.0,...,0.0,0.0,1.5,3.0,2.0,2.0,3.0,1.5,7.0,5.0
3,00cdab1a,Saturday June 14,56.2,43.8,20.0,8.0,7.0,1.0,8.0,4.0,...,0.0,0.0,0.8,1.8,0.0,0.0,1.8,0.8,7.0,1.0
4,0182f52b,Saturday July 19,51.5,48.5,17.0,10.0,4.0,4.0,3.0,3.0,...,0.0,0.0,2.0,0.9,0.0,0.0,0.9,2.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,ff30d11e,Saturday March 9 2024,59.2,40.8,8.0,11.0,5.0,6.0,3.0,2.0,...,0.0,0.0,1.6,1.3,0.0,2.0,1.3,1.6,5.0,6.0
998,ff46f6c3,Saturday June 22 2024,49.6,50.4,9.0,16.0,3.0,8.0,1.0,3.0,...,0.0,0.0,2.5,1.7,1.0,2.0,1.7,2.6,3.0,8.0
999,ff4ea1b4,Saturday May 17,44.7,55.3,6.0,11.0,3.0,4.0,0.0,3.0,...,0.0,0.0,0.4,0.9,4.0,0.0,0.9,0.4,3.0,4.0
1000,ffb5ce29,Saturday July 13 2024,62.2,37.8,17.0,9.0,7.0,0.0,4.0,7.0,...,0.0,0.0,0.6,2.0,1.0,0.0,2.0,0.6,7.0,0.0


In [19]:
## find any na values
nasum = final.isna().sum()

In [20]:
nasum

match_id                          0
date                              0
home_team                         0
away_team                         0
general_aerial_duels_won_home    10
                                 ..
shooting_off_target_away          9
shooting_on_target_away           9
xg_shots_on_target_away           9
xg_shots_away                     9
xg_total_team_xg_away            10
Length: 120, dtype: int64

In [21]:
final.to_csv('../../data_files/cleaned/matches/teams/mls_match_team_stats_reframed.csv', index=False)

In [22]:
wide = pd.read_csv('../../data_files/cleaned/matches/teams/mls_match_team_stats_reframed.csv')

In [23]:
wide.head()

,match_id,date,home_team,away_team,general_aerial_duels_won_home,general_blocked_shots_home,general_blocked_home,general_clean_sheets_home,general_clearances_home,general_corners_home,...,possession_76_80_away,possession_81_85_away,possession_86_90_away,shooting_blocked_away,shooting_goals_away,shooting_off_target_away,shooting_on_target_away,xg_shots_on_target_away,xg_shots_away,xg_total_team_xg_away
0,FridayJuly25_Columbus_Orlando,Friday July 25,Columbus,Orlando,6.0,4.0,4.0,0.0,6.0,9.0,...,61.16,43.51,34.91,2.0,3.0,2.0,6.0,6.0,10.0,2.0
1,FridayJuly25_Dallas_NewYorkCity,Friday July 25,Dallas,New York City,14.0,1.0,1.0,0.0,3.0,2.0,...,40.05,74.32,41.92,3.0,4.0,1.0,5.0,5.0,9.0,1.7
2,FridayJuly25_LAFC_Portland,Friday July 25,LAFC,Portland,8.0,4.0,4.0,0.0,3.0,4.0,...,35.72,16.70,55.20,1.0,1.0,3.0,4.0,4.0,8.0,0.4
3,FridayJuly25_NewEngland_Montréal,Friday July 25,New England,Montréal,8.0,9.0,9.0,0.0,8.0,3.0,...,42.86,23.66,32.48,3.0,3.0,2.0,5.0,5.0,11.0,1.5
4,FridayJuly25_SanDiego_Nashville,Friday July 25,San Diego,Nashville,10.0,2.0,2.0,0.0,5.0,2.0,...,45.27,33.57,42.54,3.0,0.0,1.0,4.0,4.0,9.0,0.6


In [25]:
month_map = {
        "january": "01", "february": "02", "march": "03", "april": "04",
        "may": "05", "june": "06", "july": "07", "august": "08",
        "september": "09", "october": "10", "november": "11", "december": "12"
    }

wide['date'] = wide['date'].str.replace(r"^[A-Za-z]+ ", "", regex=True)

wide['date'] = wide['date'].apply(
        lambda x: re.sub(
            r"([A-Za-z]+)",
            lambda m: month_map[m.group(1).lower()],
            x
        )
    )

wide['date'] = wide['date'].apply(lambda x: x + " 2025" if re.search(r"\d{4}$", x) is None else x)

wide['date'] = pd.to_datetime(wide['date'], format="%m %d %Y")

In [26]:
wide.head()

,match_id,date,home_team,away_team,general_aerial_duels_won_home,general_blocked_shots_home,general_blocked_home,general_clean_sheets_home,general_clearances_home,general_corners_home,...,possession_76_80_away,possession_81_85_away,possession_86_90_away,shooting_blocked_away,shooting_goals_away,shooting_off_target_away,shooting_on_target_away,xg_shots_on_target_away,xg_shots_away,xg_total_team_xg_away
0,FridayJuly25_Columbus_Orlando,2025-07-25,Columbus,Orlando,6.0,4.0,4.0,0.0,6.0,9.0,...,61.16,43.51,34.91,2.0,3.0,2.0,6.0,6.0,10.0,2.0
1,FridayJuly25_Dallas_NewYorkCity,2025-07-25,Dallas,New York City,14.0,1.0,1.0,0.0,3.0,2.0,...,40.05,74.32,41.92,3.0,4.0,1.0,5.0,5.0,9.0,1.7
2,FridayJuly25_LAFC_Portland,2025-07-25,LAFC,Portland,8.0,4.0,4.0,0.0,3.0,4.0,...,35.72,16.70,55.20,1.0,1.0,3.0,4.0,4.0,8.0,0.4
3,FridayJuly25_NewEngland_Montréal,2025-07-25,New England,Montréal,8.0,9.0,9.0,0.0,8.0,3.0,...,42.86,23.66,32.48,3.0,3.0,2.0,5.0,5.0,11.0,1.5
4,FridayJuly25_SanDiego_Nashville,2025-07-25,San Diego,Nashville,10.0,2.0,2.0,0.0,5.0,2.0,...,45.27,33.57,42.54,3.0,0.0,1.0,4.0,4.0,9.0,0.6


In [27]:
wide.to_csv('../../data_files/cleaned/matches/teams/mls_match_team_stats_reframed.csv', index=False)